In [1]:
from directory_cobveg_load import definir_diretorio 
project_path = definir_diretorio()

import geopandas as gpd
import os

from utils import(
    get_data_diretorio,
)
from utils.downloads import(
    download_malha_geosampa,
)

# Cobertura Vegetal 2020 (GeoSampa)

In [2]:
data_path = get_data_diretorio(project_path)

## Importar dados 

In [3]:
cache_path = os.path.join(
    data_path, 
    'cache', 
    'cob_veg_concat.parquet'
)
if os.path.exists(cache_path):
    print('Carregando arquivo em cache')
    gdf_cobertura_vegetal = gpd.read_parquet(cache_path)
else:
    gdf_cobertura_vegetal = download_malha_geosampa(
        'cobertura_vegetal', 
        project_path, 
        True
    )

http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=geoportal:cobertura_vegetal&maxFeatures=2000&startIndex=0
cobertura_vegetal_pg1
(2000, 19)
http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=geoportal:cobertura_vegetal&maxFeatures=2000&startIndex=2000
cobertura_vegetal_pg2
(2000, 19)
http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=geoportal:cobertura_vegetal&maxFeatures=2000&startIndex=4000
cobertura_vegetal_pg3
(2000, 19)
http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=geoportal:cobertura_vegetal&maxFeatures=2000&startIndex=6000
cobertura_vegetal_pg4
(2000, 19)
http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFor

In [4]:
gdf_cobertura_vegetal.shape

(272117, 19)

In [5]:
gdf_cobertura_vegetal.sample(3)

,cd_identif,cd_identi0,cd_control,tx_datum,cd_coorden,cd_coorde0,cd_categor,cd_subcate,cd_subcat0,tx_descric,qt_area_ve,qt_perimet,qt_altura_,qt_desvio_,tx_data_vo,tx_entrega,tx_lote,tx_lote_ve,geometry
1555,91556.0,91752.0,64268.0,"SIRGAS 2000,4 - 23S",335028.05,7405583.84,13.0,0.0,0.0,"Média a alta cobertura arbórea, arbóreo-arbust...",253.22943,101.21744,0.0,0.0,2017,07_2018,006,V4,"POLYGON ((335015.304 7405591.298, 335015.849 7..."
44,188045.0,188526.0,180047.0,"SIRGAS 2000,4 - 23S",328614.43,7380000.77,13.0,0.0,0.0,"Média a alta cobertura arbórea, arbóreo-arbust...",60.91158,29.25177,0.0,0.0,2017,09_2018,009,V2,"POLYGON ((328616.542 7380003.475, 328618.048 7..."
484,92485.0,92682.0,63153.0,"SIRGAS 2000,4 - 23S",334999.16,7405198.99,13.0,0.0,0.0,"Média a alta cobertura arbórea, arbóreo-arbust...",2753.93829,438.26008,0.0,0.0,2017,07_2018,006,V4,"POLYGON ((334968.607 7405224.701, 334968.529 7..."
